Producer

In [1]:
import datetime
import random
import time

In [ ]:
from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

In [ ]:
import pandas as pd
columns_of_interest = [
    'lpep_pickup_datetime',
    'lpep_dropoff_datetime',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'tip_amount',
     
]
df_green = pd.read_csv('green_tripdata_2019-10.csv.gz', compression='gzip', usecols=columns_of_interest) #, error_bad_lines=False)
df_green.head()

In [ ]:
topic_name = 'green-trips'
time0 = time.time()
# tqdm.tqdm(df_green.head().iterrows())
for _, row in tqdm.tqdm(df_green.iterrows(),len(df_green)) :
    row_dict = row.to_dict()
    row_dict['timestamp'] = datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
    time.sleep(random.randrange(0, 100) / 1000)
    producer.send(topic_name, value=row_dict)
    # print(row_dict)
    
producer.flush()
t1 = time.time()
print(f'took {(t1 - time0):.2f} seconds')

Consumer

In [ ]:
import pyspark
from pyspark.sql import SparkSession

pyspark_version = pyspark.__version__
kafka_jar_package = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{pyspark_version}"

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .config("spark.jars.packages", kafka_jar_package) \
    .getOrCreate()

In [ ]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [ ]:
from pyspark.sql import types

schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType()) \
    .add("timestamp", types.StringType())


Analytics

In [ ]:
from pyspark.sql import functions as F

green_stream = green_stream \
  .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
  .select("data.*")

In [ ]:
popular_destinations = green_stream \
    .groupBy(F.window(F.col("timestamp"), "5 minutes"),'DOLocationID') \
    .agg(F.count("*").alias("total_count")) \
    .orderBy("total_count", ascending=False)

In [ ]:
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

query.awaitTermination()